In [1]:
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit.quantum_info import Operator
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from framework import Framework

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

seed = 24593457
np.random.seed(seed=seed)
rng=np.random.default_rng(seed=seed)

In [2]:
### perm ansatz
# [1] Nicola Mariella, Andrea Simonetto, *A Quantum Algorithm for the Sub-Graph Isomorphism Problem*, https://arxiv.org/abs/2111.09732#
# [2] https://github.com/qiskit-community/subgraph-isomorphism

from typing import Union, Tuple
from qiskit.circuit import ParameterVector

IntOrTuple = Union[int, Tuple]

def _hcph(phi: float, qc: QuantumCircuit, qubits: IntOrTuple):
    ctrl, target = qubits if isinstance(qubits, tuple) else (-1, qubits)
    qc.h(target)
    if ctrl >= 0:
        qc.cp(phi, ctrl, target)
    else:
        qc.p(phi, target)
    qc.h(target)


S4_BLOCK_PARCOUNT = 5


def _s4_block(params: np.ndarray) -> QuantumCircuit:
    """Build the basic block S4 for the permutations Ansatz.

    Args:
        params (np.ndarray): The array of parameters.
    """
    params = np.asarray(params).flatten()
    assert params.shape == (S4_BLOCK_PARCOUNT,)
    qc = QuantumCircuit(QuantumRegister(name="q", size=2))
    _hcph(params[0], qc, 0)

    qc.h(1)
    qc.p(params[1], 1)
    qc.cp(params[2], 0, 1)
    qc.h(1)

    _hcph(params[3], qc, (1, 0))
    _hcph(params[4], qc, (0, 1))
    return qc


def _map_qreg(array, qreg: QuantumRegister) -> np.ndarray:
    assert isinstance(qreg, QuantumRegister)
    array = np.asarray(array)
    fun = np.vectorize(lambda i: qreg[i])
    return fun(array)


def _expand_topology(topology, *, qreg: QuantumRegister) -> np.ndarray:
    if isinstance(topology, str):
        if topology in {"linear", "circular"}:
            # v = np.arange(qreg.size - 1)
            # v = np.stack([v, v + 1]).T
            v = np.array([[qreg.size - 1, 0]])
            for i in range(qreg.size):
                for j in range(qreg.size):
                    if i < j:
                        v = np.concatenate([v, np.array([[i, j]])], axis=0)
            # if topology == "circular" and qreg.size > 2:
            #     v = np.concatenate([v, np.array([[qreg.size - 1, 0]])], axis=0)
            topology = v
        else:
            raise ValueError(f"Unrecognized topology: {topology}")
    topology = np.asarray(topology)
    assert topology.ndim == 2 and topology.shape[1] == 2
    return _map_qreg(topology, qreg=qreg)

def params_tensor(shape, *, name="t") -> np.ndarray:
    """Prepare a tensor of circuit parameters."""
    shape = tuple(np.atleast_1d(shape).flatten())
    v = ParameterVector(name=name, length=np.product(shape))
    v = np.array(v.params)
    return v.reshape(shape)

def s4_ansatz(
    topology, *, qreg: Union[QuantumRegister, int], params=None
) -> Tuple[QuantumCircuit, np.ndarray]:
    """Construct the permutations ansatz based on the S4 block.

    Args:
        topology (str, np.ndarray): The topology for the ansatz, see the function
        ansatz() for more information.
        qreg (QuantumRegister, int): The destination quantum register.
        params (np.ndarray): The array of parameters.
    """
    if isinstance(qreg, int):
        qreg = QuantumRegister(qreg)
    assert isinstance(qreg, QuantumRegister)
    topology = _expand_topology(topology, qreg=qreg)
    if params is None:
        params = params_tensor((len(topology), S4_BLOCK_PARCOUNT))
    params = np.asarray(params)
    assert params.ndim == 2 and params.shape[1] == S4_BLOCK_PARCOUNT
    assert len(params) == len(topology)
    qc = QuantumCircuit(qreg)
    for v, q in zip(params, topology):
        qc.compose(_s4_block(v), qubits=q, inplace=True)

    qc_ = QuantumCircuit(qreg)
    qc_.compose(qc.to_gate(label="PermAnsatz"), inplace=True)
    return qc_, params

def thetas_to_prob(x) -> np.ndarray:
    x = np.asarray(x) / np.pi
    x = np.abs(x)
    r = np.modf(x)
    r = r[0], r[1] % 2
    return np.abs(r[0] - r[1])

def sample_exact_thetas(v, *, n=1, seed=None):
    if isinstance(v, dict):
        dkeys = v.keys()
        v = np.array(list(v.values()))
    v = thetas_to_prob(v)
    rng = np.random.default_rng(seed)
    prob = rng.uniform(size=(n, len(v)))
    v = (prob < v) * np.pi
    if dkeys is not None:
        v = [dict(zip(dkeys, v1)) for v1 in v]
        assert len(v) == n
    return v

In [3]:
def Classic_Solve(adj):
    N = len(adj)
    M = 1 << N
    dp = np.ones((N, M)) * -np.inf
    next = np.zeros((N, M))

    for i in range(N):
        dp[i][1 << i] = 0
        next[i][1 << i] = -1
    
    for k in range(0, M):
        for i in range(N):
            if ((k >> i) & 1 != 1):
                continue
            for t in range(N):
                if (k >> t) & 1 != 1:
                    continue
                if adj[i][t] == 0:
                    continue
                if dp[i][k] < adj[i][t] + dp[t][k ^ (1 << i)]:
                    dp[i][k] = adj[i][t] + dp[t][k ^ (1 << i)]
                    next[i][k] = t
    
    mx = 0
    mx_i = 0
    mx_j = 0
    for i in range(N):
        for j in range(M):
            if mx < dp[i][j]:
                mx = dp[i][j]
                mx_j = j
                mx_i = i
    
    tour = []
    while mx_i != -1:
        tour.append(mx_i)
        next_i = int(next[mx_i][mx_j])
        mx_j = mx_j ^ (1 << mx_i)
        mx_i = next_i

    return mx, tour

In [4]:
def RandomPermutationMatrix(n):
  p = np.arange(n) + 1
  np.random.shuffle(p)
  m = []
  for i in range(n):
    line = np.zeros(n).astype(int)
    line[p[i] - 1] = 1
    m.append(line)
  return np.array(m)

In [5]:
def RandomDirectedGraph(n):
    g = nx.DiGraph()
    for i in range(n):
        for j in range(n):
            if i != j:
                g.add_edge(i, j, weight=np.random.randint(20) - 10)

    adj_g = nx.adjacency_matrix(g).todense()
    nx_ans, order = Classic_Solve(adj_g)
    return adj_g, nx_ans, order

In [6]:
def solveLP(n, adj, get_info):
    biggest_number = np.max(adj)
    adj[adj==0] = -biggest_number * (n-1)
    for i in range(n):
        adj[i, i] = 0
    adj = np.append(adj, np.zeros(n * n).reshape(n, n), axis=1)
    down = np.ones(n * n).reshape(n, n) * -biggest_number * (n-1)
    adj = np.append(adj, np.append(down, np.zeros(n * n).reshape(n, n),axis=1), axis=0)
    
    min_number = np.min(adj)
    adj[:,:] -= min_number

    n = n << 1
    qbit = int(np.log2(n))
    qc, params = s4_ansatz('circular', qreg=qbit)
    ansatz = QuantumCircuit(qbit * 2)
    ansatz.compose(qc, inplace=True, qubits=range(qbit))
    ansatz.compose(qc, inplace=True, qubits=np.array(range(qbit)) + qbit)

    a = np.zeros(n * n)
    for i in range(int(n/2) - 1):
        a[i * n + i + 1] = 1

    f = Framework()
    f.encode(adj.flatten().astype(int))
    f.operation(a)
    f.set_ansatz(ansatz)

    qubits_number = 0
    depth = 0
    if get_info:
        qc = f.loss_qc()
        qc = transpile(qc, basis_gates=['cx', 'u1', 'u2', 'u3'])
        qubits_number = len(qc.qubits)
        depth = qc.depth()

    initial_point = (rng.uniform(size=len(qc.parameters)) - 1/2) * np.pi
    result = f.Run(seed=seed, init_point=initial_point, max=True)

    qc1 = s4_ansatz('circular', qreg=int(np.log2(len(adj))), params=params)[0]
    sampled_params_dicts = sample_exact_thetas(result.optimal_parameters,
                                                n=32, seed=seed)
    max_cost = -999999
    for v in sampled_params_dicts:
        p1 = np.abs(Operator(qc1.bind_parameters(v)).data)
        p1 = np.round(p1)
        cost = ((p1 @ (adj + min_number) @ p1.T) * a.reshape(n, n)).sum()
        if cost > max_cost:
            max_cost = cost
            p2 = p1
    
    return max_cost, p2, (qubits_number, depth)

In [7]:
def RandomDirectedGraphTest(n, RandomPermu=100):
    adj_g, nx_ans, _ = RandomDirectedGraph(n)

    all_cost = []
    for i in range(RandomPermu):
        P = RandomPermutationMatrix(n)
        adj = P @ adj_g @ P.T
        cost, p, info = solveLP(n, adj, False)
        if i == 0:
            ret_info = info
        all_cost.append([cost, p.flatten()])

    return nx_ans, all_cost, adj_g, ret_info


In [8]:
ans, all_cost, adj_g, info = RandomDirectedGraphTest(8, 100)

quantum_ans = np.array(all_cost)[:, 0]

print(adj_g)
print("answer: ", ans)
print("min: ", np.min(quantum_ans))
print("max: ", np.max(quantum_ans))
print("mean: ", np.mean(quantum_ans))
print("info: ", info)

with open('./LP/8-random-10-test100', 'a+') as f:
  print("seed = ", seed, file=f)
  print(adj_g, file=f)
  print("info: ", info, file=f)
  print("answer: ", ans, file=f)
  print("min: ", np.min(quantum_ans), file=f)
  print("max: ", np.max(quantum_ans), file=f)
  print("mean: ", np.mean(quantum_ans), file=f)
  print(quantum_ans, file=f)
  print(all_cost, file=f)

[[  0  -9   7  -6  -8  -6   9  -7]
 [  7   0   6 -10   8   2  -5  -2]
 [ -3   4   0  -1  -7   6   3   8]
 [ -9  -7  -9   0   7  -4  -3   8]
 [ -3  -7   2  -4   0  -6  -4  -1]
 [  4   4  -4  -3  -2   0  -1   9]
 [  7  -8  -7  -7   6  -2   0 -10]
 [ -1  -1   9  -9  -9  -4   8   0]]
answer:  49.0
min:  0.0
max:  34.0
mean:  10.31
info:  (0, 0)


/var/folders/3b/54p896jd5hz9r642m7ngjx7h0000gn/T/ipykernel_6140/2525240444.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  quantum_ans = np.array(all_cost)[:, 0]
